# Analyzing results from notebooks

The `.ipynb` format is capable of storing tables and charts in a standalone file. This makes it a great choice for model evaluation reports. `NotebookCollection` allows you to retrieve results from previously executed notebooks to compare them.

In [1]:
import papermill as pm
import jupytext

from sklearn_evaluation import NotebookCollection

Let's first generate a few notebooks, we have a `train.py` script that trains a single model, let's convert it to a jupyter notebook:

In [2]:
nb = jupytext.read('train.py')
jupytext.write(nb, 'train.ipynb')

We use papermill to execute the notebook with different parameters, we'll train 4 models: 2 random forest, a linear regression and a support vector regression:

In [4]:
# models with their corresponding parameters
params = [{
    'model': 'sklearn.ensemble.RandomForestRegressor',
    'params': {
        'n_estimators': 50
    }
}, {
    'model': 'sklearn.ensemble.RandomForestRegressor',
    'params': {
        'n_estimators': 100
    }
}, {
    'model': 'sklearn.linear_model.LinearRegression',
    'params': {}
}, {
    'model': 'sklearn.svm.LinearSVR',
    'params': {}
}]

# ids to identify each experiment
ids = [
    'random_forest_1', 'random_forest_2', 'linear_regression',
    'support_vector_regression'
]

# output files
files = [f'{i}.ipynb' for i in ids]

# execute notebooks using papermill
for f, p in zip(files, params):
    pm.execute_notebook('train.ipynb', output_path=f, parameters=p)

Executing:   0%|          | 0/17 [00:00<?, ?cell/s]

Executing:   0%|          | 0/17 [00:00<?, ?cell/s]

Executing:   0%|          | 0/17 [00:00<?, ?cell/s]

Executing:   0%|          | 0/17 [00:00<?, ?cell/s]

To use `NotebookCollection`, we pass a a list of paths, and optionally, ids for each notebook (uses paths by default).

The only requirement is that cells whose output we want to extract must have tags, each tag then becomes a key in the notebook collection. For instructions on adding tags, [see this](https://papermill.readthedocs.io/en/latest/usage-parameterize.html).

Extracted tables add colors to certain cells to identify the best and worst metrics. By default, it assumes that metrics are errors (smaller is better). If you are using scores (larger is better), pass `scores=True`, if you have both, pass a list of scores:

In [5]:
nbs = NotebookCollection(paths=files, ids=ids, scores=['r2'])

To get a list of tags available:

In [6]:
list(nbs)

['model_name', 'feature_names', 'model_params', 'plot', 'metrics', 'houseage']

`model_params` contains a dictionary with model parameters, let's get them (click on the tabs to switch):

In [7]:
# pro-tip: then typing the tag, press the "Tab" key for autocompletion!
nbs['model_params']

`plot` has a `y_true` vs `y_pred` chart:

In [8]:
nbs['plot']

On each notebook, `metrics` outputs a data frame with a single row with mean absolute error (mae) and mean squared error (mse) as columns.

For single-row tables, a "Compare" tab shows all results at once:

In [9]:
nbs['metrics']

We can see that the second random forest is performing the best in both metrics.

`houseage` contains a multi-row table where with error metrics broken down by the `HouseAge` indicator feature. Multi-row tables *do not* display the "Compare" tab:

In [17]:
nbs['houseage']

,mae,mse,r2
HouseAge,,,
1.0,0.767700,0.589364,NaN
2.0,0.599907,0.710334,0.595768
3.0,0.479741,0.350220,0.462628
4.0,0.455627,0.443980,0.575692
5.0,0.378937,0.300442,0.611565
6.0,0.462348,0.703149,0.363620
7.0,0.361878,0.281907,0.312370
8.0,0.343787,0.227829,0.822608
9.0,0.305901,0.242166,0.696575


If we only compare two notebooks, the output is a bit different:

In [18]:
# only compare two notebooks
nbs_two = NotebookCollection(paths=files[:2], ids=ids[:2], scores=['r2'])

Comparing single-row tables includes a diff column with the error difference between experiments. Error reductions are showed in green, increments in red:

In [19]:
nbs_two['metrics']

When comparing multi-row tables, the "Compare" tab appears, showing the difference between the tables:

In [20]:
nbs_two['houseage']

,mae,mse,r2
HouseAge,,,
1.000000,0.153380,0.259025,nan
2.000000,-0.009220,-0.064914,0.036941
3.000000,-0.015520,-0.031304,0.048032
4.000000,-0.041182,-0.067965,0.064954
5.000000,0.022155,0.021691,-0.028045
6.000000,-0.016610,0.039650,-0.035886
7.000000,-0.006585,-0.018315,0.044674
8.000000,-0.015568,-0.024139,0.018795
9.000000,-0.012251,-0.034225,0.042883


When displaying dictionaries, a "Compare" tab shows with a diff view:

In [21]:
nbs_two['model_params']

f,1,{,f,1,{
,2,"'bootstrap': True,",,2,"'bootstrap': True,"
,3,"'ccp_alpha': 0.0,",,3,"'ccp_alpha': 0.0,"
,4,"'criterion': 'squared_error',",,4,"'criterion': 'squared_error',"
,5,"'max_depth': None,",,5,"'max_depth': None,"
,6,"'max_features': 1.0,",,6,"'max_features': 1.0,"
,7,"'max_leaf_nodes': None,",,7,"'max_leaf_nodes': None,"
,8,"'max_samples': None,",,8,"'max_samples': None,"
,9,"'min_impurity_decrease': 0.0,",,9,"'min_impurity_decrease': 0.0,"
,10,"'min_samples_leaf': 1,",,10,"'min_samples_leaf': 1,"
,11,"'min_samples_split': 2,",,11,"'min_samples_split': 2,"


Lists (and sets) are compared based on elements existence:

In [16]:
nbs_two['feature_names']

Both,Only in random_forest_1,Only in random_forest_2
AveBedrms,,
AveOccup,,
AveRooms,,
HouseAge,,
Latitude,,
Longitude,,
MedInc,,
Population,,


## Using the mapping interface

`NotebookCollection` has a dict-like interface, you can retrieve data from individual notebooks:

In [ ]:
nbs['model_params']['random_forest_1']

In [ ]:
nbs['plot']['random_forest_2']